# Introduction

THis notebook contains he final notebook submission to the Kaggle competion predcit Calories challenge. I will start by first havoing code to the best possible solution based on the public leaderboard) and then explore other options. EDA and other stuff have been removed from this notebook for readability.

In [1]:
# imports

import pandas as pd
import numpy as np

In [2]:
train = pd.read_csv('PredictCalorie/train.csv')
test = pd.read_csv('PredictCalorie/test.csv')
sample = pd.read_csv('PredictCalorie/sample_submission.csv')

# Use of CatBoost

In [16]:
# Define columns to exclude from features
exclude_cols = ['id', 'Calories']
target_col = 'Calories'

# Split features and target from train
X_train = train.drop(columns=exclude_cols)
y_train = train[target_col]

# Prepare test set (same feature columns)
X_test = test.drop(columns=['id'])

# Identify categorical features (assumes object types are categorical)
categorical_features = ['Sex']

# Optional: split validation set from train (for early stopping)
X_tr, X_val, y_tr, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Define Pools
train_pool = Pool(X_tr, y_tr, cat_features=categorical_features)
val_pool = Pool(X_val, y_val, cat_features=categorical_features)
test_pool = Pool(X_test, cat_features=categorical_features)

# Train the model
model = CatBoostRegressor(
    iterations=2000,
    learning_rate=0.03,
    depth=10,
    random_strength=5,
    l2_leaf_reg=7,
    bootstrap_type='Bernoulli',
    grow_policy='SymmetricTree',
    eval_metric='RMSE',
    random_seed=42,
    early_stopping_rounds=50,
    verbose=100
)

model.fit(train_pool, eval_set=val_pool)

# Predict on test set
test_preds = model.predict(test_pool)

# Add predictions to test dataframe
test['Calories'] = test_preds

# Optional: preview results
print(test[['id','Calories']].head())
test[['id','Calories']].to_csv('catboost.csv',index=False)

0:	learn: 60.6811318	test: 60.5082800	best: 60.5082800 (0)	total: 36.1ms	remaining: 1m 12s
100:	learn: 5.7573990	test: 5.7426415	best: 5.7426415 (100)	total: 1.49s	remaining: 28.1s
200:	learn: 3.8117644	test: 3.8401566	best: 3.8401566 (200)	total: 3.02s	remaining: 27s
300:	learn: 3.6295186	test: 3.6791427	best: 3.6791427 (300)	total: 4.77s	remaining: 26.9s
400:	learn: 3.5529231	test: 3.6217363	best: 3.6217363 (400)	total: 6.49s	remaining: 25.9s
500:	learn: 3.5078001	test: 3.5929045	best: 3.5929045 (500)	total: 8.19s	remaining: 24.5s
600:	learn: 3.4760466	test: 3.5780044	best: 3.5780044 (600)	total: 9.86s	remaining: 23s
700:	learn: 3.4526859	test: 3.5672550	best: 3.5672550 (700)	total: 11.6s	remaining: 21.5s
800:	learn: 3.4337295	test: 3.5609999	best: 3.5609999 (800)	total: 13.3s	remaining: 19.9s
900:	learn: 3.4170465	test: 3.5564010	best: 3.5564010 (900)	total: 15s	remaining: 18.3s
1000:	learn: 3.4020588	test: 3.5525224	best: 3.5525209 (999)	total: 16.8s	remaining: 16.8s
1100:	learn: 3

In [74]:
from sklearn.model_selection import StratifiedKFold
import pandas as pd
import numpy as np

# Bin 'Duration' into quantile-based categories (e.g., 5 groups)
X_train = train.drop(columns=exclude_cols)
y_train = train[target_col]

# Add Duration binning for stratification
duration_bins = pd.qcut(train['Duration'], q=10, labels=False)  # or use pd.cut for fixed bins

# Initialize StratifiedKFold using duration bins
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

test_preds = np.zeros(len(test))


# Proceed with the same logic — just replace KFold with StratifiedKFold
for fold, (idx_train, idx_valid) in enumerate(cv.split(X_train, duration_bins)):
    print(f'\nFold {fold + 1}')
    
    X_tr, X_val = X_train.iloc[idx_train], X_train.iloc[idx_valid]
    y_tr, y_val = np.log1p(y_train.iloc[idx_train]), np.log1p(y_train.iloc[idx_valid])
    
    # CatBoost Pools
    train_pool = Pool(X_tr, y_tr, cat_features=categorical_features)
    val_pool = Pool(X_val, y_val, cat_features=categorical_features)
    test_pool = Pool(X_test, cat_features=categorical_features)
    
    # Train and predict as before...
    model = CatBoostRegressor(
        iterations=2000,
        learning_rate=0.03,
        depth=10,
        random_strength=5,
        l2_leaf_reg=7,
        bootstrap_type='Bernoulli',
        grow_policy='SymmetricTree',
        eval_metric='RMSE',
        random_seed=42,
        early_stopping_rounds=50,
        verbose=100
    )
    
    model.fit(train_pool, eval_set=val_pool)
    test_preds += np.expm1(model.predict(test_pool))

test_preds /= cv.get_n_splits()
test['Calories'] = test_preds
test[['id','Calories']].to_csv('catboost_5_fold_str.csv',index=False)


Fold 1
0:	learn: 0.9366100	test: 0.9354519	best: 0.9354519 (0)	total: 33.6ms	remaining: 1m 7s
100:	learn: 0.0924376	test: 0.0926483	best: 0.0926483 (100)	total: 1.57s	remaining: 29.5s
200:	learn: 0.0637060	test: 0.0646417	best: 0.0646417 (200)	total: 3.27s	remaining: 29.2s
300:	learn: 0.0606240	test: 0.0618747	best: 0.0618747 (300)	total: 5.35s	remaining: 30.2s
400:	learn: 0.0595081	test: 0.0610308	best: 0.0610305 (399)	total: 7.21s	remaining: 28.7s
500:	learn: 0.0588160	test: 0.0605886	best: 0.0605886 (500)	total: 9.31s	remaining: 27.8s
600:	learn: 0.0583254	test: 0.0603792	best: 0.0603782 (599)	total: 11s	remaining: 25.7s
700:	learn: 0.0579711	test: 0.0602938	best: 0.0602938 (700)	total: 12.8s	remaining: 23.7s
800:	learn: 0.0576959	test: 0.0602223	best: 0.0602223 (800)	total: 14.7s	remaining: 22s
900:	learn: 0.0574552	test: 0.0601871	best: 0.0601871 (900)	total: 16.4s	remaining: 20s
1000:	learn: 0.0572292	test: 0.0601481	best: 0.0601469 (991)	total: 18.4s	remaining: 18.4s
1100:	lear

# Add more features

In [43]:
train = pd.read_csv('PredictCalorie/train.csv')
test = pd.read_csv('PredictCalorie/test.csv')
sample = pd.read_csv('PredictCalorie/sample_submission.csv')
train.head()

,id,Sex,Age,Height,Weight,Duration,Heart_Rate,Body_Temp,Calories
0,0,male,36,189.0,82.0,26.0,101.0,41.0,150.0
1,1,female,64,163.0,60.0,8.0,85.0,39.7,34.0
2,2,female,51,161.0,64.0,7.0,84.0,39.8,29.0
3,3,male,20,192.0,90.0,25.0,105.0,40.7,140.0
4,4,female,38,166.0,61.0,25.0,102.0,40.6,146.0


In [44]:
train['bmi']=train['Weight']/(train['Height']*train['Height'])*100*100
test['bmi']=test['Weight']/(test['Height']*test['Height'])*100*100

train['dur_body_heart']=train['Duration']*train['Heart_Rate']*train['Body_Temp']
test['dur_body_heart']=test['Duration']*test['Heart_Rate']*test['Body_Temp']



In [48]:
def calculate_bmr(row):
    if row['Sex'].lower() == 'male':
        return 10 * row['Weight'] + 6.25 * row['Height'] - 5 * row['Age'] + 5
    elif row['Sex'].lower() == 'female':
        return 10 * row['Weight'] + 6.25 * row['Height'] - 5 * row['Age'] - 161
    else:
        return np.nan  # handle unknown or missing Sex

train['BMR'] = train.apply(calculate_bmr, axis=1)
test['BMR'] = test.apply(calculate_bmr, axis=1)

In [49]:
train.head()

,id,Sex,Age,Height,Weight,Duration,Heart_Rate,Body_Temp,Calories,bmi,dur_body_heart,BMR
0,0,male,36,189.0,82.0,26.0,101.0,41.0,150.0,22.955684,107666.0,1826.25
1,1,female,64,163.0,60.0,8.0,85.0,39.7,34.0,22.582709,26996.0,1137.75
2,2,female,51,161.0,64.0,7.0,84.0,39.8,29.0,24.690405,23402.4,1230.25
3,3,male,20,192.0,90.0,25.0,105.0,40.7,140.0,24.414062,106837.5,2005.00
4,4,female,38,166.0,61.0,25.0,102.0,40.6,146.0,22.136740,103530.0,1296.50


# XGB Regressor

In [79]:
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import TargetEncoder
import xgboost as xgb
import numpy as np
import pandas as pd

categorical_cols = ['Sex']

# Discretize 'duration' into 5 bins for stratification
duration_bins = pd.qcut(X['Duration'], q=5, labels=False, duplicates='drop')

# Use StratifiedKFold for stratified splitting based on duration bins
cv = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)
y_pred = np.zeros(len(test))

# Log-transform the target
y_log = np.log1p(y)  # log(1 + y) for RMSLE-friendly training

for idx_train, idx_valid in cv.split(X, duration_bins):
    X_train, y_train = X.iloc[idx_train].copy(), y_log.iloc[idx_train]
    X_valid, y_valid = X.iloc[idx_valid].copy(), y_log.iloc[idx_valid]
    X_test = test[X.columns].copy()

    # Target Encoding for categorical column
    encoder = TargetEncoder(random_state=42)
    X_train['Sex'] = encoder.fit_transform(X_train[['Sex']], y_train)[:, 0]
    X_valid['Sex'] = encoder.transform(X_valid[['Sex']])[:, 0]
    X_test['Sex'] = encoder.transform(X_test[['Sex']])[:, 0]

    model = xgb.XGBRegressor(
        n_estimators=2000,
        max_depth=10,
        learning_rate=0.03,
        subsample=0.7,
        colsample_bytree=0.7,
        objective='reg:squarederror',  # L2 loss
        eval_metric='rmse',
        early_stopping_rounds=100,
        random_state=42,
        verbosity=1,
    )

    model.fit(
        X_train, y_train,
        eval_set=[(X_valid, y_valid)],
        verbose=100
    )

    y_pred += np.expm1(model.predict(X_test))

# Average predictions over folds
y_pred /= cv.get_n_splits()

# Create submission file
pred_xgb = y_pred
sample_submission = pd.read_csv('sample_submission.csv')
submission_xgb = pd.DataFrame({'id': sample_submission.id, 'Calories': pred_xgb})
submission_xgb.to_csv('submission_xgb.csv', index=False)

[0]	validation_0-rmse:0.93585
[100]	validation_0-rmse:0.09281
[200]	validation_0-rmse:0.06295
[300]	validation_0-rmse:0.06119
[400]	validation_0-rmse:0.06072
[500]	validation_0-rmse:0.06057
[600]	validation_0-rmse:0.06059
[615]	validation_0-rmse:0.06061
[0]	validation_0-rmse:0.93293
[100]	validation_0-rmse:0.09143
[200]	validation_0-rmse:0.06255
[300]	validation_0-rmse:0.06088
[400]	validation_0-rmse:0.06040
[500]	validation_0-rmse:0.06028
[600]	validation_0-rmse:0.06030
[653]	validation_0-rmse:0.06032
[0]	validation_0-rmse:0.93667
[100]	validation_0-rmse:0.09201
[200]	validation_0-rmse:0.06195
[300]	validation_0-rmse:0.06014
[400]	validation_0-rmse:0.05969
[500]	validation_0-rmse:0.05959
[600]	validation_0-rmse:0.05961
[629]	validation_0-rmse:0.05962
[0]	validation_0-rmse:0.93631
[100]	validation_0-rmse:0.09176
[200]	validation_0-rmse:0.06168
[300]	validation_0-rmse:0.05986
[400]	validation_0-rmse:0.05933
[500]	validation_0-rmse:0.05916
[600]	validation_0-rmse:0.05915
[652]	validation

# ENsemble the two best cases (In log space)

In [87]:
data1=pd.read_csv('submission_xgb (1).csv')
data2=pd.read_csv('catboost_5_fold_str (1).csv')

data = data1.merge(data2, on='id', suffixes=('_xgb', '_cat'))

In [88]:
data.head()

,id,Calories_xgb,Calories_cat
0,750000,27.488697,27.206506
1,750001,107.910565,107.878140
2,750002,87.331006,87.098264
3,750003,126.202942,125.542912
4,750004,76.272044,76.180985


In [89]:
import pandas as pd
import numpy as np



# 1. Simple average of the two predictions
data['Calories_avg'] = (data['Calories_xgb'] + data['Calories_cat']) / 2

# 2. Log-scale average (RMSLE-style averaging)
data['Calories_log_avg'] = np.expm1(
    (np.log1p(data['Calories_xgb']) + np.log1p(data['Calories_cat'])) / 2
)

print(data[['id', 'Calories_xgb', 'Calories_cat', 'Calories_avg', 'Calories_log_avg']].head())

       id  Calories_xgb  Calories_cat  Calories_avg  Calories_log_avg
0  750000     27.488697     27.206506     27.347601         27.347250
1  750001    107.910565    107.878140    107.894353        107.894351
2  750002     87.331006     87.098264     87.214635         87.214558
3  750003    126.202942    125.542912    125.872927        125.872498
4  750004     76.272044     76.180985     76.226515         76.226501


In [90]:
# Save simple average with 'Calories' as the second column
data[['id', 'Calories_avg']].rename(columns={'Calories_avg': 'Calories'}).to_csv('submission_reg_avg.csv', index=False)

# Save log-scale average with 'Calories' as the second column
data[['id', 'Calories_log_avg']].rename(columns={'Calories_log_avg': 'Calories'}).to_csv('submission_log_avg.csv', index=False)

# Start fresh with new features

In [3]:
train = pd.read_csv('PredictCalorie/train.csv')
test = pd.read_csv('PredictCalorie/test.csv')
sample = pd.read_csv('PredictCalorie/sample_submission.csv')



In [ ]:
train['bmi']=train['Weight']/(train['Height']*train['Height'])*100*100
test['bmi']=test['Weight']/(test['Height']*test['Height'])*100*100

train['dur_body_heart']=train['Duration']*train['Heart_Rate']*train['Body_Temp']
test['dur_body_heart']=test['Duration']*test['Heart_Rate']*test['Body_Temp']

def calculate_bmr(row):
    if row['Sex'].lower() == 'male':
        return 10 * row['Weight'] + 6.25 * row['Height'] - 5 * row['Age'] + 5
    elif row['Sex'].lower() == 'female':
        return 10 * row['Weight'] + 6.25 * row['Height'] - 5 * row['Age'] - 161
    else:
        return np.nan  # handle unknown or missing Sex

train['BMR'] = train.apply(calculate_bmr, axis=1)
test['BMR'] = test.apply(calculate_bmr, axis=1)

# Squares of features
train['dur_2']=train['Duration']*train['Duration']
test['dur_2']=test['Duration']*test['Duration']

train['hr_2']=train['Heart_Rate']*train['Heart_Rate']
test['hr_2']=test['Heart_Rate']*test['Heart_Rate']

# Square root of Duration
train['dur_sqrt'] = train['Duration'] ** 0.5
test['dur_sqrt'] = test['Duration'] ** 0.5

train['Intensity'] = train['Heart_Rate'] / train['Duration']
test['Intensity'] = test['Heart_Rate'] / test['Duration']

In [4]:
train.head()

,id,Sex,Age,Height,Weight,Duration,Heart_Rate,Body_Temp,Calories
0,0,male,36,189.0,82.0,26.0,101.0,41.0,150.0
1,1,female,64,163.0,60.0,8.0,85.0,39.7,34.0
2,2,female,51,161.0,64.0,7.0,84.0,39.8,29.0
3,3,male,20,192.0,90.0,25.0,105.0,40.7,140.0
4,4,female,38,166.0,61.0,25.0,102.0,40.6,146.0


In [5]:
train.describe()

,id,Age,Height,Weight,Duration,Heart_Rate,Body_Temp,Calories
count,750000.000000,750000.000000,750000.000000,750000.000000,750000.000000,750000.000000,750000.000000,750000.000000
mean,374999.500000,41.420404,174.697685,75.145668,15.421015,95.483995,40.036253,88.282781
std,216506.495284,15.175049,12.824496,13.982704,8.354095,9.449845,0.779875,62.395349
min,0.000000,20.000000,126.000000,36.000000,1.000000,67.000000,37.100000,1.000000
25%,187499.750000,28.000000,164.000000,63.000000,8.000000,88.000000,39.600000,34.000000
50%,374999.500000,40.000000,174.000000,74.000000,15.000000,95.000000,40.300000,77.000000
75%,562499.250000,52.000000,185.000000,87.000000,23.000000,103.000000,40.700000,136.000000
max,749999.000000,79.000000,222.000000,132.000000,30.000000,128.000000,41.500000,314.000000


In [6]:
train.Duration.value_counts()

Duration
9.0     28900
26.0    28768
19.0    28433
15.0    28084
11.0    27967
25.0    27765
5.0     27493
28.0    27239
16.0    27215
8.0     26743
10.0    26723
14.0    26701
3.0     26153
21.0    26109
6.0     26102
4.0     25969
7.0     25061
12.0    24790
27.0    24758
24.0    24749
17.0    24611
13.0    24249
29.0    23856
20.0    23775
2.0     23689
18.0    23418
22.0    23208
23.0    22360
1.0     12648
30.0    12464
Name: count, dtype: int64

In [7]:
len(train)

750000

In [8]:
len(train.Duration.unique())

30

In [10]:
len(train.Age.unique())

60